## NDVI Raster Collection from Google Earth Engine
This document provides a step-by-step guide to collecting NDVI (Normalized Difference Vegetation Index) raster data using Google Earth Engine (GEE). NDVI is a widely used index for assessing vegetation health and coverage.

In [ ]:
#!pip install ee
#!pip install geemap

  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Obtaining dependency information for blessings from https://files.pythonhosted.org/packages/03/74/489f85a78247609c6b4f13733cbf3ba0d864b11aa565617b645d6fdf2a4a/blessings-1.7-py3-none-any.whl.metadata
  Created wheel for ee: filename=ee-0.2-py3-none-any.whl size=3706 sha256=faed7dc40608d4886ef9bb3c7f6bcaea408d05a37d1f48dc558e66eee18dc713
  Stored in directory: c:\users\astau\appdata\local\pip\cache\wheels\cb\b1\a5\d16d395d190f232181add4c4af67a74c2de272875759921aef
Successfully built ee


## Import nessecary modules

In [4]:
import ee
import geemap
import geopandas as gpd

c:\Users\astau\anaconda3\envs\geospatial\lib\site-packages\google\api_core\_python_version_support.py:266: FutureWarning: You are using a Python version (3.10.12) which Google will stop supporting in new releases of google.api_core once it reaches its end of life (2026-10-04). Please upgrade to the latest Python version, or at least Python 3.11, to continue receiving updates for google.api_core past that date.
  warnings.warn(message, FutureWarning)


In [ ]:
# Initially I had to authenticate the API, but now this step is not necessary. Showing for replication purposes. 
ee.Authenticate()
ee.Initialize()

True

An automated notebook message will provide steps on how to authenticate the Google Earth Engine API.

In [34]:
# Load Philly boundary file
phillygdf = gpd.read_file("../../data/City_Limits.geojson")

# Convert to Earth Engine geometry
philly_ee = geemap.geopandas_to_ee(phillygdf)

In [41]:
def get_philly_ndvi():
    landsat8 = ee.ImageCollection("LANDSAT/LC08/C02/T1_L2") \
        .filterBounds(philly_ee) \
        .filterDate('2024-06-01', '2024-08-31') \
        .filter(ee.Filter.lt('CLOUD_COVER', 10))  # Relaxed to 15%
    
    def mask_clouds(image):
        qa = image.select('QA_PIXEL')
        # Both bits should be 0 (clear)
        cloud_bit = 1 << 3
        shadow_bit = 1 << 4
        
        # Create masks (True = clear pixel)
        clear_mask = qa.bitwiseAnd(cloud_bit).eq(0) \
                       .And(qa.bitwiseAnd(shadow_bit).eq(0))  # Fixed!
        
        return image.updateMask(clear_mask)
    
    composite = landsat8.map(mask_clouds).median()
    ndvi = composite.normalizedDifference(['SR_B5', 'SR_B4']).rename('NDVI')
    
    return ndvi.clip(philly_ee)

# Re-run
ndvi_image = get_philly_ndvi()

# Check it has data before exporting
# This will fail if no data:
try:
    test_value = ndvi_image.sample(ee.Geometry.Point([-75.16, 39.95]), 30).first().get('NDVI').getInfo()
    print(f"✅ NDVI has data! Sample value: {test_value}")
except:
    print("❌ NDVI has no data - check your code")


# Export NDVI image to GeoTIFF
task = ee.batch.Export.image.toDrive(
    image=ndvi_image,
    description='Philly_NDVI_Summer2024_fixed',
    folder='EarthEngineExports',
    fileNamePrefix='philly_ndvi_2024_summer_fixed',
    scale=30,
    crs='EPSG:2272',
    maxPixels=1e13
)

task.start()

print("✅ Export started with working NDVI!")
print("Monitor at: https://code.earthengine.google.com/tasks")
print(f"Task ID: {task.id}")



✅ NDVI has data! Sample value: 0.06864574551582336
✅ Export started with working NDVI!
Monitor at: https://code.earthengine.google.com/tasks
Task ID: MKOJAWYTHJAUSVRSVQR6VG24


## Resources
GEE Python Installation Documents - https://developers.google.com/earth-engine/guides/python_install
GEE Community Tutorials - https://developers.google.com/earth-engine/community/tutorials
Opendataphilly 